# 3 Additional Functions with Keras

- 3.1 모델 합치기 with CIFAR10
- 3.2 데이터 증강
- 3.3 Finetuning

## 3.1 모델 합치기 with CIFAR10

convolution과 maxpooling layer로 구성된 feature extractor 모델과

fully connected layer로 구성된 ANN classifier 모델을 따로 정의하고

두 모델을 합쳐서 CNN 모델을 만듬

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(4)

### (1) 데이터셋 : CIFAR 10

In [ ]:
import tensorflow.keras.utils as utils
from tensorflow.keras import datasets

# Dataset Load
(X_train, Y_train), (X_test, Y_test) = datasets.cifar10.load_data()

# Dataset Confirm
print(X_train.shape, Y_train.shape)
print('label : ',Y_train[0])
plt.imshow(X_train[0])

# Dataset Preprocessing
X_train = X_train/255.0
X_test = X_test/255.0
Y_train = utils.to_categorical(Y_train)
Y_test = utils.to_categorical(Y_test)

print(X_train.shape, Y_train.shape)

### (2) 모델링

In [3]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Flatten, BatchNormalization, Dropout, ReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
n_in = X_train.shape[1:]
n_out = Y_train.shape[-1]

def conv_maxpool_layers(n_in):
    # Coding Time
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3,3), padding='same', activation='relu', input_shape=(n_in)))
    model.add(Conv2D(32, kernel_size=(3,3), padding='same', strides=(2,2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def fc_layers(n_out):
    # Coding Time
    model = Sequential()
    model.add(Dense(units=128, input_shape = (2048,), activation='relu'))
    model.add(Dense(units=n_out, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def CNN_sum(n_in, n_out):
    
    # Coding Time
    
    #각 부분 모델
    feature_extractor=conv_maxpool_layers(n_in)
    feature_extractor.trainable=True
    ann_classifier=fc_layers(n_out)
    ann_classifier.trainable=True
    
    #두 모델을 합쳐 새로운 모델 정의(Functional Style)
    x= Input(shape=n_in)
    feature = feature_extractor(x)
    y= ann_classifier(feature)
    model = Model(inputs = x, outputs = y)
    
    '''
    Sequential Style
    model = Sequential()
    model.add(feature_extractor)
    model.add(ann_classifier)
    '''
    return model

In [ ]:
model = CNN_sum(n_in, n_out)
model.summary()

### (3-4) 모델의 학습과정 설정 / 모델 학습

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping
earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='auto', restore_best_weights=True)
history = model.fit(X_train, Y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks = [earlystopper])

### (5) 모델 평가

In [ ]:
loss_and_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))

## 3.2 Image data augmentation
케라스에서는 ImageDataGenerate 함수로 Data augmentation 기능을  제공

https://keras.io/preprocessing/image/#imagedatagenerator-class

### (1)-2 데이터 증강 적용

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 2, # 회전
    zoom_range = 0.1, # 확대 축소
    width_shift_range = 0.1, # 수평 이동
    height_shift_range = 0.1, # 수직 이동
    horizontal_flip = True, # 수평 반전|
    vertical_flip = False # 수직 반전
)

datagen.fit(X_train)

### (3-4) 모델의 학습과정 설정 / 모델 학습

In [ ]:
model = CNN_sum(n_in, n_out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='auto', restore_best_weights=True)
# Coding Time
model.fit(datagen.flow(X_train[:-10000], Y_train[:-10000], batch_size= 128),
         epochs = 50, validation_data=(X_train[-10000:],Y_train[-10000:]), verbose = 1)


In [ ]:
loss_and_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))

## 3.3 Transfer learning
Transfer learning을 통해 현재 쓰이고 있는 네트워크를 가져와 학습하는 방법을 배워본다(Classifier만 / Entire)

### (2) 모델링1 : Classifier learning
    
사용가능 네트워크 :
https://keras.io/api/applications/

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
# Coding Time
base_model = VGG16(weights='imagenet', input_shape=(32,32,3), include_top=False)
base_model.summary()

In [ ]:
# Get Feature Extroactor from VGG16
x = base_model.output
# 1,1,512
# Add Classifier
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(Y_train.shape[1], activation='softmax')(x) #Y_train.shape[1] :10 

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers: # Frosen 하기
    layer.trainable = False

In [ ]:
model.summary()

### (3) 모델의 학습과정 설정

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### (4) 모델 학습시키기

In [ ]:
earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='auto', restore_best_weights=True)
history = model.fit(X_train, Y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks = [earlystopper])

In [ ]:
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

### (5) 모델 평가하기

In [ ]:
loss_and_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))

### (2) 모델링1 : Entire
    
사용가능 네트워크 :
https://keras.io/api/applications/

In [ ]:
# Get Feature Extroactor from VGG16
x = base_model.output

# Add Classifier
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(Y_train.shape[1], activation='softmax')(x) #Y_train.shape[1] :10 

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# first: train the all layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
model.summary()

### (3) 모델의 학습과정 설정

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### (4) 모델 학습시키기

In [ ]:
earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='auto', restore_best_weights=True)
history = model.fit(X_train, Y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks = [earlystopper])

In [ ]:
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

### (5) 모델 평가하기

In [ ]:
loss_and_accuracy = model.evaluate(X_test, Y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))

# 3.4 Keras MNIST - 모델의 성능을 직접 높혀보자
 - CNN의 구조를 바꾸어 나만의 모델을 만들어보자
 - 목표 정확도: 99.5% 만들기
 - 바꿀 수 있는 하이퍼 파라미터: Learning Rate, Batch size, Epochs, Optimizer, Activation Function, 모델 레이어 구조, BN, DO, DA, Fine Tuning 등

## (1) 데이터셋

In [2]:
# Coding Time
(x_train, y_train),(x_test, y_test) = datasets.mnist.load_data()
print(x_train.shape, y_train.shape)

NameError: name 'datasets' is not defined

## (2) Keras 모델링

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 2, # 회전
    zoom_range = 0.1, # 확대 축소
    width_shift_range = 0.1, # 수평 이동
    height_shift_range = 0.1, # 수직 이동
    horizontal_flip = True, # 수평 반전|
    vertical_flip = False # 수직 반전
)

datagen.fit(X_train)

In [ ]:
n_in = X_train.shape[1:]
n_out = Y_train.shape[-1]

def conv_maxpool_layers(n_in):
    # Coding Time
    model = Sequential()
    model.add(Conv2D(16, kernel_size=3, padding='same', activation='relu', input_shape=(n_in)))
    model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def fc_layers(n_out):
    # Coding Time
    model = Sequential()
    model.add(Dense(units=14*14, input_shape = (14*14,1), activation='relu'))
    model.add(Dense(units=n_out, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def CNN_sum(n_in, n_out):
    
    # Coding Time
    
    #각 부분 모델
    feature_extractor=conv_maxpool_layers(n_in)
    feature_extractor.trainable=True
    ann_classifier=fc_layers(n_out)
    ann_classifier.trainable=True
    
    #두 모델을 합쳐 새로운 모델 정의(Functional Style)
    x= Input(shape=n_in)
    feature = feature_extractor(x)
    y= ann_classifier(feature)
    model = Model(inputs = x, outputs = y)
    
    '''
    Sequential Style
    model = Sequential()
    model.add(feature_extractor)
    model.add(ann_classifier)
    '''
    return model

In [ ]:
model = CNN_sum(n_in, n_out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='auto', restore_best_weights=True)
history = model.fit(x_train, y_train, batch_size=128, epochs=50, validation_split=0.2, callbacks = [earlystopper])

In [ ]:
loss_and_accuracy = model.evaluate(x_test, y_test, batch_size=128)
print('loss : %.4f, accruracy : %.4f'%(loss_and_accuracy[0],loss_and_accuracy[1]))